In [ ]:
import pyaudio, sys, socket

In [ ]:
#This block shows the devices available to pyaudio
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

In [ ]:
def recvall(sock,buff):
    #This function creates a buffer to ensure packets of the correct length are fed in the algorithm
    
    #sock is the socket object
    #buff is the size of the buffer
    packet = sock.recv(buff)
    if len(packet) < buff:
        ext = sock.recv(buff-len(packet))
        packet += ext
    return packet

In [ ]:
#This is the Server code that allows for the transmission of multichannel audio data from the respeaker 
#It was created by gazzwi86 at https://www.raspberrypi.org/forums/viewtopic.php?t=23504


port = 5000
chunk = 512
FORMAT = pyaudio.paInt16
CHANNELS = 6
RATE = 16000

#creates pyaudio object to use audio input/output devices
p = pyaudio.PyAudio()
stream = p.open(format = FORMAT, channels = CHANNELS, rate = RATE, input = True, output = True, frames_per_buffer = chunk)

#creates socket object 
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('', port)) # listen on port 5000
server_socket.listen(1) # queue max 5 connections
client_socket, address = server_socket.accept()

print "Your IP address is: ", socket.gethostbyname(socket.gethostname())
print "Server Waiting for client on port ", port

#transmits the
while True:   
    try:
        client_socket.sendall(stream.read(chunk))
    except IOError,e:
        if e[1] == pyaudio.paInputOverflowed: 
            print e 
            x = '\x00'*16*256*2 #value*format*chunk*nb_channels 

stream.stop_stream()
stream.close()
socket.close()
p.terminate()


In [ ]:
import pyaudio, sys, socket
import numpy as np

#This is the client code that receives the audio data transmitted by the 6 channels of the Respeaker
#and writes it in the default output device
#It is a modification of the code made by gazzwi86 at https://www.raspberrypi.org/forums/viewtopic.php?t=23504

port = 5000
ip = "put the ip of the server here"

chunk = 256
FORMAT = pyaudio.paInt16
CHANNELS = 1
channels = 6
RATE = 16000

#creates pyaudio object
p = pyaudio.PyAudio()
stream = p.open(format = FORMAT, channels = CHANNELS, rate = RATE, input = False,output = True, frames_per_buffer = chunk)

#Create a socket connection for connecting to the server:
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect((ip, port))

while True:

    #Receive data from the server and store them in a channelxchunk*2 list of numpy arrays 
    b = bytearray()
    data = recvall(client_socket,2*channels*chunk)
    numpydata = np.frombuffer(data, dtype=np.int16)
    numpydata = np.reshape(numpydata, (chunk, 6))
    numpydata = [numpydata[:,i] for i in range(channels)]
    data = [numpydata[i].astype(np.int16).tobytes() for i in range(channels)]
    stream.write(data[0])

socket.close()